In [13]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, TimeoutError, LocalCluster, progress
from dask.diagnostics import ProgressBar
import pandas as pd
import hvplot.dask
import re
from textblob import TextBlob
from langdetect import detect
from langdetect import detect_langs

#code found on stack exchange
#makes a new client if one is not running, runs 4 workers
try:
    client = Client('tcp://localhost:8787', timeout='5s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=6, ip='localhost')
    client = Client(cluster)


client


<Client: 'tcp://127.0.0.1:8787' processes=6 threads=12, memory=16.00 GiB>

In [14]:
keyWords = [
    'mental health', 
    'depression',
    'depressed', 
    'anxiety',
    'anxious', 
    'ADHD', 
    'attention deficit',
    'OCD',
    'obsessive-compulsive disorder',
    'PTSD',
    'posttraumatic stress disorder',
    'trauma',
    'bipolar',
    'autism',
    'schizophrenia',
    'eating disorder',
    'anorexia',
    'bulimia',
    'psychosis',
    'psychologist',
    'psychotherapist',
    'psychiatrist',
    'psychotherapy',
    'depressant'
]

In [15]:
#contains tweets (and time of the tweet) that contain mental health key word
#polled every 2 minutes
tweetdf = dd.read_csv(urlpath='mental_health_tweets.csv', sep='\t' )

#contains counts of tweets for every hour
countdf = dd.read_csv(urlpath='mental_health_tweet_count.csv', sep='\t')

#try reading in the preprocessed tweets
# if the csv does not exist, create it
try:
    # read the preprocessed csv
    tweetdf = dd.read_parquet(path='mental_health_tweets_preprocessed_en.parquet')
except:
    #contains tweets (and time of the tweet) that contain mental health key word
    #polled every 2 minutes
    tweetdf = dd.read_csv(urlpath='mental_health_tweets.csv', sep='\t')
    
    #pre process
    # preprocess instructions found at https://youtu.be/ujId4ipkBio
    def preprocess(text):
        # remove mentions
        text = re.sub('@[a-zA-Z0-9]+', '', text)
        
        #  remove hashtag symbol
        text = re.sub('#', '', text)
        
        # remove retweet identifier
        text = re.sub('RT[\s]+', '', text)
        
        # remove links
        text = re.sub('https?:\/\/\S+', '', text)


        text = re.sub('[^a-zA-Z\\s]', '', text)

        text = text.strip()

        return text

    tweetdf['text'] = tweetdf['tweet'].map(preprocess)

    # remove any tweets that are either empty, or not in english
    tweetdf = tweetdf.loc[tweetdf['text'] != '']
    tweetdf['language'] = tweetdf['text'].map(lambda x: detect(x))
    tweetdf = tweetdf.loc[tweetdf['language'] == 'en']

    # drop original tweet and language columns
    tweetdf = tweetdf.drop(columns=['tweet', 'language'])

    #write pre processed text to parquet (dask recommends parquet)
    dd.to_parquet(tweetdf, path='mental_health_tweets_preprocessed_en.parquet')
    
    

In [16]:

#specify column types
tweetdf['time'] = dd.to_datetime(tweetdf['time'])
countdf['timestamp'] = dd.to_datetime(countdf['timestamp'])

#separate the components of the time for grouping
tweetdf['year'] = tweetdf['time'].dt.year
tweetdf['month'] = tweetdf['time'].dt.month
tweetdf['day'] = tweetdf['time'].dt.day
tweetdf['hour'] = tweetdf['time'].dt.hour
tweetdf['minute'] = tweetdf['time'].dt.minute

countdf['year'] = countdf['timestamp'].dt.year
countdf['month'] = countdf['timestamp'].dt.month
countdf['day'] = countdf['timestamp'].dt.day
countdf['hour'] = countdf['timestamp'].dt.hour

In [17]:
#sort the dataframes based on ascening tme
countdf = countdf.sort_values(ascending=True, by=['month', 'day', 'hour'])
tweetdf = tweetdf.sort_values(ascending=True, by=['month', 'day', 'hour', 'minute'])

#make a display string for the x axis of the graph
countdf['ts_display'] = countdf['month'].astype('str') + '/' + countdf['day'].astype('str') + '\t' + countdf['hour'].astype('str') + ':00'

# countdf = client.persist(countdf)

In [18]:
# countdf.hvplot(x='ts_display', y='count').opts(xrotation=90, fontscale=0.5)

In [19]:
#determine key words in tweet
def containsKey(text, key):
    return text.str.contains(key)

# create a new column for each key, with values of whether or not
# the tweet in that row contains that key
for i in keyWords:
    tweetdf[i] = tweetdf['text'].map_partitions(containsKey, i)


In [20]:
#count tweets each keyword appears in
#if a tweet contains multiple keywords, then they are added 
#to the result df as |word1|word2| etc.

def keysInTweet(df):
    keys = ''
    for k in keyWords:
        if df[k]:
            keys = keys + '|' + k
    keys = keys + '|'
    return keys

tweetdf['keys'] = tweetdf.apply(keysInTweet, axis=1, meta=('keys', 'object'))

# convert the value counts to its own df
multiKeyCount = tweetdf['keys'].value_counts().to_frame()
# REMOVE? drop rows that dont have a key in the tweet
multiKeyCount = multiKeyCount.loc[multiKeyCount['keys'] != '']
# copy the key column (which actually holds counts), rename it and then drop the original
multiKeyCount['count'] = multiKeyCount['keys']
multiKeyCount = multiKeyCount.drop(labels='keys', axis=1)

# multiKeyCount = client.persist(multiKeyCount)

In [21]:
# multiKeyCount.hvplot.bar(y='count', use_index=True)

In [22]:
# create an empty dask dataframe, with 1 parition (since max rows is 24)
keyCount = pd.DataFrame({'key' : keyWords})
keyCount = dd.from_pandas(keyCount, npartitions=1)

# given key k, find the number of times k appears in a tweet
def countKeys(k):
    return tweetdf[k].compute()

# create a new row of the key count
keyCount['count'] = keyCount['key'].map(countKeys, meta=('count', 'int64'))

# sort rows by descending count
keyCount = keyCount.sort_values(by='count', ascending=True)

# keyCount = client.persist(keyCount)

In [23]:
# keyCountPlot = keyCount.hvplot.bar(y='count', x='key').opts(fontscale=0.75, invert_axes=True)
# hvplot.save(keyCountPlot, 'keyCountPlot.png')

In [24]:
def compSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def compPolarity(text):
    return TextBlob(text).sentiment.polarity


tweetdf['subjectivity'] = tweetdf['text'].map(compSubjectivity)
tweetdf['polarity'] = tweetdf['text'].map(compPolarity)


tweetdf.compute()

,time,text,year,month,day,hour,minute,mental health,depression,depressed,...,bulimia,psychosis,psychologist,psychotherapist,psychiatrist,psychotherapy,depressant,keys,subjectivity,polarity
2,2022-11-13 18:20:49.359485,I think thats a common psychosis in women And ...,2022.0,11.0,13.0,18.0,20.0,False,False,False,...,False,True,False,False,False,False,False,|OCD|psychosis|,0.566667,-0.075000
3,2022-11-13 18:20:49.359485,EVERYONE GETTING AUTISM USERS AND I CANT THINK...,2022.0,11.0,13.0,18.0,20.0,False,False,False,...,False,False,False,False,False,False,False,|,0.000000,0.000000
4,2022-11-13 18:20:49.359485,of Tropical Depression Nicole pg,2022.0,11.0,13.0,18.0,20.0,False,False,False,...,False,False,False,False,False,False,False,|,0.000000,0.000000
5,2022-11-13 18:20:49.359485,What if its that adhdautistic mothers higher c...,2022.0,11.0,13.0,18.0,20.0,False,False,False,...,False,False,False,False,False,False,False,|autism|,0.666667,0.250000
6,2022-11-13 18:20:49.359485,Old Mama speaking as a longtime social worker ...,2022.0,11.0,13.0,18.0,20.0,False,False,False,...,False,False,False,False,True,False,False,|psychiatrist|,0.215278,0.005556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53903,2022-11-17 15:21:28.228904,PR Entire xc party can be split into adhd vs a...,2022.0,11.0,17.0,15.0,21.0,False,False,False,...,False,False,False,False,False,False,False,|autism|,0.625000,0.000000
53904,2022-11-17 15:21:28.228904,I ordered to on Order No OD its Totally diff...,2022.0,11.0,17.0,15.0,21.0,False,True,False,...,False,False,False,False,False,False,False,|depression|,0.600000,0.000000
51487,NaT,Wrong The cause of death needs to be attribute...,NaN,NaN,NaN,NaN,NaN,False,False,False,...,False,False,False,False,False,False,False,|trauma|,0.950000,-0.250000
51488,NaT,but not related to the,NaN,NaN,NaN,NaN,NaN,False,False,False,...,False,False,False,False,False,False,False,|,0.400000,0.000000
